## Triangular Arbitrage Calculation Based on Historical Exchange Rates

### Overview:

Triangular arbitrage involves exploiting discrepancies between three related currency pairs. In this case, we focus on the pairs **EUR/USD**, **USD/JPY**, and **EUR/JPY**. The goal is to identify situations where the calculated cross rate for **EUR/JPY** (derived from **EUR/USD** and **USD/JPY**) differs from the actual **EUR/JPY** market rate, potentially indicating an arbitrage opportunity.

### Formula for Calculating Cross Rate:

To calculate the cross rate for **EUR/JPY**, we multiply the exchange rates of **EUR/USD** and **USD/JPY**:

$$
\text{Calculated\_EURJPY} = \text{EUR/USD} \times \text{USD/JPY}
$$

Where:
- **EUR/USD** is the exchange rate for euros to US dollars.
- **USD/JPY** is the exchange rate for US dollars to Japanese yen.
- **Calculated\_EURJPY** represents the theoretical cross rate for **EUR/JPY** based on the two other rates.

### Arbitrage Detection:

Once we have the **calculated EUR/JPY** value, we compare it to the actual **EUR/JPY** exchange rate:

$$
\text{Arbitrage\_Opportunity} = \text{YES} \quad \text{if} \quad \left| \text{Calculated\_EURJPY} - \text{Actual\_EURJPY} \right| > 0.01
$$

Where:
- **Calculated\_EURJPY** is the cross rate derived from **EUR/USD** and **USD/JPY**.
- **Actual\_EURJPY** is the observed market exchange rate for **EUR/JPY**.
- If the absolute difference between the **calculated** and **actual** EUR/JPY rates is greater than 0.01, it is flagged as an arbitrage opportunity.

### Data Collection:

We fetch historical exchange rates for the following currency pairs:
1. **EUR/USD**
2. **USD/JPY**
3. **EUR/JPY**

These rates are collected over a period of one year. The historical data is used to identify potential arbitrage opportunities for each day in the past year.

### Example:

Let’s assume we have the following rates for a specific date:
- **EUR/USD = 1.12**
- **USD/JPY = 110.50**
- **Actual EUR/JPY = 123.62**

We calculate the **EUR/JPY cross rate** as follows:

$$
\text{Calculated\_EURJPY} = 1.12 \times 110.50 = 123.76
$$

The actual EUR/JPY rate is **123.62**, so we compute the difference:

$$
\left| 123.76 - 123.62 \right| = 0.14
$$

Since the difference is greater than **0.01**, this would be flagged as an **arbitrage opportunity**.

                    ---------------------------------------------------------


### Okay!! Lets analyse the historical exchange rates to find arbitrage opportunites appeared in the past.


In [20]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

def fetch_historical_data(currency_pair, start_date, end_date):
    """
    Fetch historical exchange rate data using Yahoo Finance for the given currency pair.
    
    Parameters:
    currency_pair (str): The currency pair symbol (e.g., 'EURUSD=X', 'GBPUSD=X').
    start_date (str): Start date for fetching historical data (format: 'YYYY-MM-DD').
    end_date (str): End date for fetching historical data (format: 'YYYY-MM-DD').
    
    Returns:
    pd.DataFrame: DataFrame containing historical exchange rates.
    """
    currency_pair = currency_pair + "=X" if not currency_pair.endswith("=X") else currency_pair
    ticker = yf.Ticker(currency_pair)
    data = ticker.history(start=start_date, end=end_date)
    
    if data.empty or 'Close' not in data.columns:
        print(f"Error fetching data for {currency_pair}")
        return pd.DataFrame()  # Return an empty DataFrame if data is not available
    
    return data[['Close']].rename(columns={'Close': currency_pair})  # Rename the 'Close' column to the currency pair name

def calculate_triangular_arbitrage(date, eur_usd, usd_jpy, eur_jpy):
    """
    Calculate whether there is an arbitrage opportunity on a given day.
    
    Parameters:
    date (str): The date of the exchange rates.
    eur_usd (float): EUR/USD exchange rate.
    usd_jpy (float): USD/JPY exchange rate.
    eur_jpy (float): EUR/JPY exchange rate.
    
    Returns:
    dict: A dictionary containing the date and whether an arbitrage opportunity exists.
    """
    # Calculate the EUR/JPY cross rate from EUR/USD and USD/JPY
    calculated_eur_jpy = eur_usd * usd_jpy
    
    # Check for arbitrage opportunity (if actual EUR/JPY is different from calculated)
    arbitrage_opportunity = "YES" if abs(calculated_eur_jpy - eur_jpy) > 0.01 else "NO"
    
    return {
        "Date": date,
        "Calculated_EURJPY": calculated_eur_jpy,
        "Actual_EURJPY": eur_jpy,
        "Opportunity": arbitrage_opportunity
    }

def list_past_arbitrage_opportunities():
    """
    List arbitrage opportunities for the past year based on historical exchange rates.
    """
    # Define the currency pairs involved in the triangular arbitrage
    pairs = ['EURUSD', 'USDJPY', 'EURJPY']
    
    # Set the start and end dates for the past year
    end_date = datetime.today().strftime('%Y-%m-%d')
    start_date = (datetime.today() - timedelta(days=365)).strftime('%Y-%m-%d')
    
    # Fetch historical data for each currency pair
    print("Fetching historical data...")
    eur_usd_data = fetch_historical_data('EURUSD', start_date, end_date)
    usd_jpy_data = fetch_historical_data('USDJPY', start_date, end_date)
    eur_jpy_data = fetch_historical_data('EURJPY', start_date, end_date)
    
    # Ensure that the data is available for all currency pairs
    if eur_usd_data.empty or usd_jpy_data.empty or eur_jpy_data.empty:
        print("Failed to fetch data for one or more pairs. Exiting.")
        return
    
    # Join the data on the date index to align exchange rates
    print("Joining historical data...")
    data = eur_usd_data.join([usd_jpy_data, eur_jpy_data], how='inner')

    print(f"Joined Data (first few rows):")
    print(data.head())  # Print the first few rows of the joined data to check

    # Iterate over each row to calculate arbitrage opportunities
    print("Detecting arbitrage opportunities...")
    opportunities = []
    
    for index, row in data.iterrows():
        arbitrage_opportunity = calculate_triangular_arbitrage(
            date=index.strftime('%Y-%m-%d'),
            eur_usd=row['EURUSD=X'],
            usd_jpy=row['USDJPY=X'],
            eur_jpy=row['EURJPY=X']
        )
        opportunities.append(arbitrage_opportunity)
    
    # Convert opportunities to a DataFrame
    opportunities_df = pd.DataFrame(opportunities)

    # Save the DataFrame in-memory
    print("Saving opportunities as a DataFrame...")
    global stored_opportunities_df
    stored_opportunities_df = opportunities_df  # Save it as a global variable to be accessed later

    # Optionally print the DataFrame
    pd.set_option('display.max_rows', None)  # To display all rows
    print("Detected Arbitrage Opportunities:")
    print(opportunities_df)

    # Optionally save to a CSV file
    try:
        opportunities_df.to_csv('arbitrage_opportunities.csv', index=False)
        print("Opportunities saved to 'arbitrage_opportunities.csv'.")
    except Exception as e:
        print(f"Error saving to CSV: {e}")

if __name__ == '__main__':
    list_past_arbitrage_opportunities()


Fetching historical data...
Joining historical data...
Joined Data (first few rows):
                           EURUSD=X    USDJPY=X    EURJPY=X
Date                                                       
2023-10-10 00:00:00+01:00  1.057888  148.326996  156.909149
2023-10-11 00:00:00+01:00  1.060479  148.628006  157.598999
2023-10-12 00:00:00+01:00  1.062428  149.035995  158.328995
2023-10-13 00:00:00+01:00  1.053674  149.789993  157.811996
2023-10-16 00:00:00+01:00  1.052155  149.565994  157.354996
Detecting arbitrage opportunities...
Saving opportunities as a DataFrame...
Detected Arbitrage Opportunities:
           Date  Calculated_EURJPY  Actual_EURJPY Opportunity
0    2023-10-10         156.913283     156.909149          NO
1    2023-10-11         157.616904     157.598999         YES
2    2023-10-12         158.340050     158.328995         YES
3    2023-10-13         157.829837     157.811996         YES
4    2023-10-16         157.366665     157.354996         YES
5    2023-10-

In [21]:
stored_opportunities_df

,Date,Calculated_EURJPY,Actual_EURJPY,Opportunity
0,2023-10-10,156.913283,156.909149,NO
1,2023-10-11,157.616904,157.598999,YES
2,2023-10-12,158.340050,158.328995,YES
3,2023-10-13,157.829837,157.811996,YES
4,2023-10-16,157.366665,157.354996,YES
5,2023-10-17,157.869298,157.863007,NO
6,2023-10-18,158.341082,158.309998,YES
7,2023-10-19,157.805887,157.789001,YES
8,2023-10-20,158.587767,158.548004,YES
9,2023-10-23,158.722427,158.712006,YES


### Output:

For each day in the past year, we generate a table containing:
- The date
- The calculated EUR/JPY rate
- The actual EUR/JPY rate
- Whether or not an arbitrage opportunity exists

### Example Output Table:

| Date       | Calculated_EURJPY | Actual_EURJPY | Opportunity |
|------------|-------------------|---------------|-------------|
| 2023-10-11 | 157.61            | 157.59        | YES         |
| 2023-10-12 | 158.34            | 158.32        | YES         |
| 2023-10-13 | 157.82            | 157.81        | YES         |
| 2023-10-16 | 157.37            | 157.35        | YES         |
| 2023-10-18 | 158.34            | 158.31        | YES         |

### Conclusion:

By comparing the **calculated** cross rate of **EUR/JPY** to the **actual market rate**, we can identify days where arbitrage opportunities may have existed in the past year. This strategy allows traders to detect potential inefficiencies in the market and capitalize on them.

## Short Summary :
Analyzing **triangular arbitrage** opportunities can provide valuable insights for both **traders** and **bankers**, particularly in **foreign exchange markets** (Forex). Here's how this type of analysis can benefit them:

### 1. **Exploiting Price Inefficiencies**
Triangular arbitrage takes advantage of **inefficiencies** in the currency market. When the exchange rates between three related currency pairs are misaligned, a trader can exploit this by executing a series of trades to profit from the price difference.

- **Traders**: By detecting these arbitrage opportunities, traders can quickly execute buy and sell orders on different currency pairs to lock in risk-free profits. These opportunities are rare and usually exist for only a short period, but they offer **risk-free profit potential** when properly executed.
  
- **Bankers**: In banking, especially for **Forex desks** or **market makers**, detecting arbitrage opportunities is essential to ensure that pricing on their currency pairs is **efficient and competitive**. They can adjust their pricing algorithms to avoid offering opportunities for arbitrage to external traders, which would otherwise lead to financial losses.

### 2. **Improved Algorithmic Trading**
For **high-frequency traders** (HFTs) and algorithmic trading systems, triangular arbitrage strategies can be automated. These systems constantly scan the market for inefficiencies in currency pricing and execute trades in milliseconds. 

- **Traders**: With the use of automated scripts like the one you've developed, traders can incorporate **triangular arbitrage strategies** into their trading bots, allowing them to capture profits from price discrepancies without manual intervention.
  
- **Bankers**: Large banks and financial institutions use algorithms that look for arbitrage opportunities. By integrating this analysis into their **trading algorithms**, banks can enhance the efficiency of their trading operations, avoid offering arbitrage opportunities to competitors, and even capitalize on such opportunities when possible.

### 3. **Hedging Against Market Risk**
Arbitrage opportunities can indicate **imbalances** in the market that, if persistent, could signal upcoming volatility or market inefficiencies. Banks and traders can use this information to **hedge** their positions against unforeseen market movements.

- **Traders**: They can combine arbitrage detection with other trading strategies to **minimize risk**. For instance, if a trader holds a position in a volatile market, they can use arbitrage opportunities to hedge their exposure by offsetting losses with gains from arbitrage trades.
  
- **Bankers**: For banks, triangular arbitrage analysis can help them maintain **market-neutral positions**, ensuring that they don’t take on unwanted risk. It can also act as a safeguard to ensure that they aren't losing money through inefficiencies in their own pricing.

### 4. **Liquidity and Market Efficiency**
Triangular arbitrage plays an important role in ensuring that the **foreign exchange market remains efficient**. When arbitrage opportunities arise, traders rush to take advantage of them, which, in turn, drives prices back into balance.

- **Traders**: They benefit from a more **liquid** and **efficient market**. While they can exploit short-lived opportunities, the constant trading activity helps stabilize the currency market over time.
  
- **Bankers**: Banks benefit from **market efficiency** because it reduces volatility and unpredictability in the exchange rates. Fewer arbitrage opportunities mean more stable markets, allowing banks to focus on larger profit margins from regular trading activity and currency hedging strategies.

### 5. **Enhancing Profitability for Currency Dealers**
For currency dealers or market makers, detecting and avoiding arbitrage opportunities can directly impact profitability. If a bank's trading desk unknowingly allows arbitrage opportunities, they might incur losses when other market participants (such as algorithmic traders) capitalize on the mispricing.

- **Traders**: Triangular arbitrage allows traders to earn a **small but consistent profit** with minimal risk by identifying and acting on price discrepancies.
  
- **Bankers**: For banks and financial institutions, preventing arbitrage ensures that they aren't leaking profits by offering **unintentionally favorable exchange rates** to savvy traders. It also helps them optimize their **currency spread** (the difference between bid and ask prices), which is a primary revenue stream for banks involved in Forex trading.

### 6. **Competitive Edge in Forex Markets**
For both traders and bankers, staying ahead of arbitrage opportunities provides a **competitive edge** in the highly competitive and fast-moving foreign exchange market.

- **Traders**: They can achieve **better trade execution** and profitability by being the first to detect and exploit arbitrage opportunities, even before competitors spot them.
  
- **Bankers**: Banks can use sophisticated tools to **optimize pricing models** and provide **better liquidity** to their clients while avoiding costly arbitrage situations.

### 7. **Market Insight and Data Analytics**
Analyzing triangular arbitrage not only helps in identifying trading opportunities but also provides insights into how different currencies interact with one another. Banks and traders can derive useful metrics and **correlations between currency pairs**.

- **Traders**: They can use this information to refine their strategies, build better **forecasting models**, and increase profitability in their currency trading.
  
- **Bankers**: For banks, **data analytics** of arbitrage opportunities can improve **risk management** practices and allow them to understand the deeper mechanics of global currency markets.

### Conclusion:
By analyzing triangular arbitrage opportunities, both **traders** and **bankers** can:
- Capitalize on short-lived, low-risk profit opportunities.
- Improve their **algorithmic trading systems**.
- Ensure market prices remain **efficient** and **competitive**.
- Avoid **unintentional financial losses** due to mispricing.
- Gain **market insights** to better manage risks and enhance trading strategies.

Ultimately, the ability to detect and analyze arbitrage opportunities can offer a clear **competitive advantage** in the Forex market.

#### Triangular arbitrage exists only when **pricing discrepancies** occur between the exchange rates of three currencies, creating an opportunity to profit by exchanging one currency for another in a loop and returning to the original currency with a profit. These opportunities can appear and disappear quickly due to several factors. Let’s explore why triangular arbitrage sometimes exists and why it doesn’t at other times.

### Reasons Why Triangular Arbitrage Exists:

1. **Market Inefficiencies**:
   - Triangular arbitrage arises due to **inefficiencies in pricing** between currency pairs. These inefficiencies are often the result of differences in supply and demand across different currency markets. For example, if a currency is in high demand in one market but not another, its exchange rate might differ slightly, creating a window for arbitrage.
   - **Latency in updating exchange rates** between different currency pairs also contributes to this. As financial markets update prices continuously, there can be brief moments where related exchange rates aren’t perfectly aligned.

2. **Differences in Liquidity**:
   - Not all currency pairs have the same **liquidity** (the ease with which an asset can be bought or sold). Major currency pairs (like EUR/USD or USD/JPY) tend to have high liquidity, and their prices adjust rapidly to market conditions. However, less liquid currency pairs might experience temporary imbalances in pricing, creating arbitrage opportunities.
   - **Liquidity mismatches** between pairs can lead to triangular arbitrage, especially if one currency is more actively traded than another at a given time.

3. **Exchange Rate Fluctuations**:
   - Exchange rates fluctuate constantly due to various factors such as changes in interest rates, inflation, political events, and economic data releases. These fluctuations can sometimes create brief periods where **cross-rates** (calculated exchange rates between two currencies) deviate from what they should be, based on the individual exchange rates.
   - During high volatility or **market shocks**, pricing inefficiencies may emerge as market participants respond differently across various currency pairs, causing temporary misalignment.

4. **Latency in Data Feeds**:
   - The existence of triangular arbitrage opportunities can be due to **data latency**—the delay in the time it takes for exchange rates to be updated across different platforms or trading venues. Even if the currency prices are correct at any given moment, differences in the speed at which the prices are updated can cause arbitrage opportunities.
   - **High-frequency traders** and **algorithmic trading systems** can capitalize on these short-lived discrepancies before the markets adjust and the arbitrage opportunity disappears.

5. **Differences in Market Participants**:
   - **Market participants** include banks, institutional investors, retail traders, and hedge funds. These participants may have different trading strategies, information, and access to market data. When they act on their respective views of the market, the pressure they exert on buying and selling different currency pairs can lead to small imbalances in pricing.
   - For example, if many traders start buying EUR/USD while others sell USD/JPY, the rates between these pairs may not stay aligned with EUR/JPY, causing a triangular arbitrage opportunity.

### Reasons Why Triangular Arbitrage Disappears:

1. **Market Efficiency**:
   - In highly efficient markets, any **arbitrage opportunities** that appear are quickly exploited by traders. When many traders engage in triangular arbitrage, they cause the currency prices to move back into balance almost instantly.
   - This process is called **price convergence**. As traders buy and sell the currencies involved in the arbitrage, the exchange rates will adjust, eliminating the opportunity.
   
2. **High Liquidity**:
   - In the case of **highly liquid currency pairs**, there are large volumes of trading happening across various market participants (banks, traders, institutions), and the exchange rates update very quickly. This makes it less likely that arbitrage opportunities will exist for more than a fraction of a second.
   - For pairs like EUR/USD, USD/JPY, and EUR/JPY, which are all very liquid, arbitrage opportunities are usually detected and closed extremely quickly, especially by **high-frequency trading (HFT) algorithms**.

3. **Algorithmic and High-Frequency Trading**:
   - **Algorithmic trading systems** and **high-frequency traders** are designed to detect and exploit arbitrage opportunities within milliseconds. These traders are constantly scanning the market for price discrepancies, and they execute trades almost immediately once an arbitrage opportunity arises.
   - As these automated systems act on arbitrage, they effectively close the gap in pricing, causing the arbitrage opportunity to disappear before human traders can act.

4. **Tight Spreads and Transaction Costs**:
   - **Spreads** (the difference between the bid and ask prices) play a critical role in arbitrage. Even if there is a pricing discrepancy, if the spread is too wide or the **transaction costs** (such as fees or slippage) are too high, the arbitrage opportunity might not be profitable. In such cases, even if triangular arbitrage "exists" on paper, it doesn't present a real profit opportunity.
   - In well-functioning markets, spreads tend to be narrow for major currency pairs, making it difficult for significant arbitrage opportunities to appear without being immediately closed by other traders.

5. **Market Stabilization**:
   - In the long run, currency markets tend to stabilize, especially for major currencies. Any brief moments of mispricing are quickly corrected as market participants adjust their positions based on new information. This market stabilization can occur due to large institutional traders who trade based on fundamentals, closing out any existing arbitrage.
   - Additionally, central banks and major financial institutions that operate in the Forex market often intervene to stabilize their currencies, making arbitrage opportunities less frequent and harder to exploit.

### Summary of Why Triangular Arbitrage Exists or Disappears:

| **Exists** | **Disappears** |
|------------|----------------|
| Pricing inefficiencies due to differences in supply/demand or market shocks | Price efficiency is restored due to algorithmic and high-frequency trading |
| Differences in liquidity between currency pairs | Arbitrage is quickly exploited and removed by active traders |
| Exchange rate fluctuations or temporary market imbalance | Market liquidity and tight spreads reduce potential arbitrage profits |
| Latency in data feeds across exchanges or trading platforms | Prices converge rapidly due to the high-speed execution of trades |
| Different market participants (retail vs. institutional) acting in different currencies | Market stabilization and efficient pricing prevent arbitrage opportunities |

### Conclusion:
Triangular arbitrage is a rare and temporary phenomenon in the Forex market. It arises due to **market inefficiencies**, **latency**, and **price misalignments** between related currency pairs. However, these opportunities disappear almost as quickly as they appear due to the **efficient nature of currency markets**, **high-frequency trading**, and **price convergence** mechanisms. 

For traders, knowing when these opportunities arise can be valuable, but the ability to act quickly and efficiently is crucial to capturing profits before the opportunity vanishes.